STEPS

~~ 1. Because the dataset is more than 1 million rows, run a train_test_split to get the R2 test score and get the LB result for the best model~~

~~ 2. Get a smaller form of the dataset (about 150,000 rows) and get both a train_test_split and 5CV scores then get the LB results for the best model~~

~~ 3. If there is no difference between 1 and 2 then perform SFS on the dataset from 2~~

4. Stack the models using either LinearRegression or Ridge
5. Hyperparameter tuning on each model 5 times
6. Stack all the 54 models
7. Perform feature engineering using OpenFE on the dataset from 2
8. Perform feature selection using FFS in OpenFE on 7 for each model
9. Hyperparameter tune each model from 8 5 times
10. Stack all the 54 models from 9
11. Stack all 108 models from 5 and 9

In [1]:
import warnings
warnings.filterwarnings('ignore')

from catboost import CatBoostRegressor
from concurrent.futures import ThreadPoolExecutor
from lightgbm import LGBMRegressor

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import numpy as np
from openfe import OpenFE, tree_to_formula, transform, TwoStageFeatureSelector, ForwardFeatureSelector
import pandas as pd
from pprint import pprint

import random

from sklearn.ensemble import ExtraTreesRegressor, HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import make_scorer, r2_score
from sklearn.model_selection import cross_validate, KFold, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import time

from xgboost import XGBRegressor

pd.set_option('display.max_columns', None)

experiment_name = 'openfe_smaller_data'

In [2]:
# train = pd.read_csv('train.csv')
train = pd.read_csv('smaller_train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

In [3]:
train.shape, test.shape

((149960, 21), (745305, 20))

In [4]:
train.sample(3)

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
496,5,5,7,6,9,3,1,4,6,3,5,2,3,2,5,8,5,3,6,6,0.490
54198,8,3,6,5,4,2,5,5,3,4,3,3,8,5,6,5,3,3,6,10,0.475
82024,7,8,5,2,6,3,5,8,4,5,6,8,1,6,6,6,4,8,3,5,0.545


In [5]:
features_list = ['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement',
       'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality',
       'Siltation', 'AgriculturalPractices', 'Encroachments',
       'IneffectiveDisasterPreparedness', 'DrainageSystems',
       'CoastalVulnerability', 'Landslides', 'Watersheds',
       'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss',
       'InadequatePlanning', 'PoliticalFactors', 'FloodProbability']

In [6]:
TARGET = 'FloodProbability'

In [7]:
X = train.drop([TARGET], axis=1)
y = train[TARGET]

n_splits = 5
k5 = KFold(n_splits=n_splits, shuffle=True, random_state=5)

- Define Models list

In [8]:
# Define pipelines
knn_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsRegressor(n_neighbors=50))
])
ridge_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('nystroem', Nystroem(n_components=500, random_state=5)),
    ('ridge', Ridge(alpha=0.1, max_iter=1000, random_state=5))
])

# Manually set pipeline names
knn_pipeline.name = 'KNN'
ridge_pipeline.name = 'Nystroem Ridge'

In [9]:
models = [
    CatBoostRegressor(random_state=5, verbose=False, early_stopping_rounds=100),
    ExtraTreesRegressor(random_state=5),
    HistGradientBoostingRegressor(random_state=5),
    LinearRegression(),
    LGBMRegressor(random_state=5, n_jobs=-1),
    RandomForestRegressor(random_state=5),
    knn_pipeline,
    ridge_pipeline,
    XGBRegressor(random_state=5),
]

- Create custom evaluation function

In [10]:
def evaluate_models_cv(models, X, y, important_features, cv_split, experiment_name):
    MLA_compare = pd.DataFrame(columns=['MLA Name', 
                                        'MLA Parameters', 
                                        'MLA Train R2', 
                                        'MLA Test R2', 
                                        'MLA Test R2 Std', 
                                        'MLA Time'])
    
    def evaluate_model(alg, idx):
        if hasattr(alg, 'name'):
            MLA_name = alg.name
        else:
            MLA_name = alg.__class__.__name__
        features = important_features.get(MLA_name, [])

        # Check if the list of important features is empty
        if len(features) == 0:
            # If empty, return results with zero values
            print(f'Skipping {MLA_name} due to no important features.')
            return {
                'MLA Name': MLA_name,
                'MLA Parameters': str(alg.get_params()),
                'MLA Train R2': 0,
                'MLA Test R2': 0,
                'MLA Test R2 Std': 0,
                'MLA Time': "0 min 0.00 sec",
            }
        
        cv_results = cross_validate(alg, 
                                    X[features], 
                                    y, cv=cv_split, 
                                    scoring='r2',
                                    return_train_score=True, 
                                    n_jobs=-1)

        # Time formatting
        mean_fit_time = cv_results['fit_time'].mean()
        minutes, seconds = divmod(mean_fit_time, 60)

        # Results population
        result = {
            'MLA Name': MLA_name,
            'MLA Parameters': str(alg.get_params()),
            'MLA Train R2': cv_results['train_score'].mean(),
            'MLA Test R2': cv_results['test_score'].mean(),
            'MLA Test R2 Std': cv_results['test_score'].std(),
            'MLA Time': f"{int(minutes)} min {seconds:.2f} sec",
        }

        print(f'Done with {MLA_name}.')
        return result

    results_list = []

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(evaluate_model, alg, idx) for idx, alg in enumerate(models)]
        for future in futures:
            result = future.result()
            results_list.append(result)

    MLA_compare = pd.DataFrame(results_list)

    MLA_compare.sort_values(by=['MLA Test R2'], ascending=False, inplace=True)
    MLA_compare.to_csv(f'{experiment_name}_results.csv', index=False)

    return MLA_compare

In [11]:
def evaluate_models_test_train(models, X, y, important_features, cv_split, experiment_name):
    MLA_compare = pd.DataFrame(columns=['MLA Name', 
                                        'MLA Parameters', 
                                        'MLA Train R2', 
                                        'MLA Test R2', 
                                        'MLA Time'])
    
    def evaluate_model(alg, idx):
        if hasattr(alg, 'name'):
            MLA_name = alg.name
        else:
            MLA_name = alg.__class__.__name__

        features = important_features.get(MLA_name, [])

        # Check if the list of important features is empty
        if len(features) == 0:
            # If empty, return results with zero values
            print(f'Skipping {MLA_name} due to no important features.')
            return {
                'MLA Name': MLA_name,
                'MLA Parameters': str(alg.get_params()),
                'MLA Train R2': 0,
                'MLA Test R2': 0,
                'MLA Time': "0 min 0.00 sec",
            }

        X_train, X_test, y_train, y_test = train_test_split(X[features],
                                                            y,
                                                            test_size=0.1,
                                                            stratify=y,
                                                            shuffle=True,
                                                            random_state=5)

        start_time = time.time()
        alg.fit(X_train, y_train)
        end_time = time.time()

        # Evaluate the model
        train_score = r2_score(y_train, alg.predict(X_train))
        test_score = r2_score(y_test, alg.predict(X_test))

        # Results population
        result = {
            'MLA Name': MLA_name,
            'MLA Parameters': str(alg.get_params()),
            'MLA Train R2': train_score,
            'MLA Test R2': test_score,
            'MLA Time': f'{(end_time - start_time) / 60:.2f} min',
        }

        print(f'Done with {MLA_name}.')
        return result

    results_list = []

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(evaluate_model, alg, idx) for idx, alg in enumerate(models)]
        for future in futures:
            result = future.result()
            results_list.append(result)

    MLA_compare = pd.DataFrame(results_list)

    MLA_compare.sort_values(by=['MLA Test R2'], ascending=False, inplace=True)
    MLA_compare.to_csv(f'{experiment_name}_results.csv', index=False)

    return MLA_compare

In [12]:
baseline_features = {}

for model in models:
    if hasattr(model, 'name'):
        model_name = model.name
    else:
        model_name = model.__class__.__name__

    baseline_features[model_name] = list(X.columns)

In [13]:
%%time

baseline_models = evaluate_models_test_train(models, X, y, baseline_features, k5, f'{experiment_name}')
baseline_models

# Raw train Linear Regression score - 0.845309
# Raw train Linear Regression 5CV score - 0.844941 (0.000751)
# Smaller train Linear Regression train_test score - 0.843288
# Smaller train Linear Regression 5CV score - 0.843459 (0.001384)
# Smaller train Linear Regression w/ default OpenFE 5CV score - 

Done with LinearRegression.
Done with Nystroem Ridge.
Done with KNN.
Done with XGBRegressor.
Done with CatBoostRegressor.
Done with LGBMRegressor.
Done with HistGradientBoostingRegressor.
Done with ExtraTreesRegressor.
Done with RandomForestRegressor.
CPU times: total: 13min 32s
Wall time: 5min 43s


,MLA Name,MLA Parameters,MLA Train R2,MLA Test R2,MLA Time
0,CatBoostRegressor,"{'loss_function': 'RMSE', 'verbose': False, 'r...",0.855833,0.844292,2.49 min
3,LinearRegression,"{'copy_X': True, 'fit_intercept': True, 'n_job...",0.843541,0.843288,0.04 min
7,Nystroem Ridge,"{'memory': None, 'steps': [('scaler', Standard...",0.793352,0.792463,0.46 min
8,XGBRegressor,"{'objective': 'reg:squarederror', 'base_score'...",0.833150,0.791628,2.20 min
2,HistGradientBoostingRegressor,"{'categorical_features': None, 'early_stopping...",0.786778,0.761517,2.59 min
4,LGBMRegressor,"{'boosting_type': 'gbdt', 'class_weight': None...",0.788021,0.760530,2.50 min
1,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",1.000000,0.659301,4.24 min
5,RandomForestRegressor,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...",0.950981,0.652607,5.41 min
6,KNN,"{'memory': None, 'steps': [('scaler', Standard...",0.667465,0.651591,0.01 min


- SFS

In [ ]:
%%time

# Initialize empty dictionary for SFS features
sfs_features = {}

for model in models:
    # set name
    if hasattr(model, 'name'):
        model_name = model.name
    else:
        model_name = model.__class__.__name__

    try:
        features = baseline_features[model_name]

        # incase there is no feature that had importance, go to the next model
        if len(features) == 0:
            continue
        
        X_sfs = X[features]

        print(f'Running backward feature selection with {model_name}')

        sfs = SFS(model,
            k_features='best',
            forward=False,
            floating=False,
            scoring='r2',
            verbose=2,
            n_jobs=7,
            cv=k5)
        
        sfs = sfs.fit(X_sfs, y)

        # Get the selected features index
        selected_sfs_idx = list(sfs.k_feature_idx_)

        # Get the feature names
        selected_sfs_feats = X_sfs.columns[selected_sfs_idx]

        selected_features = list(selected_sfs_feats)

        # Reorder selected_features based on the predefined features_list
        selected_features_ordered = [feat for feat in features_list if feat in selected_features]

        sfs_features[model_name] = selected_features_ordered

        print(f'Done with {model_name}', end='\n\n')

    except KeyError:
        print(f'{model_name} not in the dictionary.')

In [ ]:
with open('sfs_features_baseline.txt', mode='w') as f:
    pprint(sfs_features, stream=f)

- Single Model Prediction

In [ ]:
model = LinearRegression()

model.fit(X, y)

In [ ]:
pred = model.predict(test)
pred

In [ ]:
pred_df = pd.DataFrame(pred, columns=[TARGET])
pred_df.head()

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission_df = pd.concat([submission['id'], pred_df], axis=1)
submission_df.head()

In [ ]:
submission_df.to_csv('submission_lr_small_data_0.843288_0.843459.csv', index=False)

- Get Stacking score

In [ ]:
%%time

meta_scores = []
meta_model = Ridge()

for i, (train_idx, meta_idx) in enumerate(k5.split(X)):
    print(f'Fold {i + 1}')
    X_train, X_meta = X.iloc[train_idx], X.iloc[meta_idx]
    y_train, y_meta = y.iloc[train_idx], y.iloc[meta_idx]

    meta_features_fold = np.zeros((X_meta.shape[0], len(models)))
    # meta_test_features = np.zeros((y.shape[0], len(models)))
    # meta_targets = np.zeros(y.shape[0])

    for i, model in enumerate(models):
        model_name = model.__class__.__name__ if not hasattr(model, 'name') else model.name
        print(f'Starting {model_name}')
        model_features = sfs_features[model_name]

        # Fit model on the selected features
        model.fit(X_train[model_features], y_train)
        preds = model.predict(X_meta[model_features])
        meta_features_fold[:, i] = preds

    # Train the meta-model on the predictions from the base models
    meta_model.fit(meta_features_fold, y_meta)
    
    # Predict using the meta-model
    final_preds = meta_model.predict(meta_features_fold)
    
    # Calculate RMSLE for the current fold
    current_fold_rmsle = rmsle(y_meta, final_preds)
    meta_scores.append(current_fold_rmsle)

### OpenFE

In [ ]:
%%time

ofe = OpenFE()
ofe.fit(
    data=X,
    label=y,
    # n_data_blocks=2,
    # feature_boosting=True,
    task='regression',
    # stage2_metric='permutation',
    # metric='rmse', 
    n_jobs=4,
)

In [ ]:
train_x, test_x = transform(X, test, ofe.new_features_list, n_jobs=4 )

In [ ]:
baseline_openfe_features = {}

for model in models:
    if hasattr(model, 'name'):
        model_name = model.name
    else:
        model_name = model.__class__.__name__

    baseline_openfe_features[model_name] = list(train_x.columns)

In [ ]:
%%time

baseline_openfe_models = evaluate_models_cv(models, X, y, baseline_openfe_features, k5, f'{experiment_name}_openfe')
baseline_openfe_models